In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import altair as alt
from altair_saver import save
import os
import re

import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df

Filename  \
0   Breaking Down Data Silos Data Mesh to Achieve ...   
1   Decentralized Data Governance as Part of a Dat...   
2   Enterprise Data Strategy A Decentralized Data ...   
3   Finding Your Way Through the Jungle of Big Dat...   
4                  01. Evolution of Data Architecture   
5           02. Terminology Data Fabric and Data Mesh   
6    03. Data Fabric and Data Mesh Use Case Scenarios   
7     04. Data Fabric and Data Mesh Business Benefits   
8      05. Key Data Fabric and Data Mesh Capabilities   
9                     06. Relevant ML and DL Concepts   
10      07. AI and ML for a Data Fabric and Data Mesh   
11                       08. AI for Entity Resolution   
12  09. Data Fabric and Data Mesh for the AI Lifec...   
13              10. Data Fabric Architecture Patterns   
14  11. Data Fabric Within an Enterprise Architecture   
15  12. Data Fabric and Data Mesh in a Hybrid Clou...   
16  13. Intelligent Cataloging and Metadata Manage...   
17    14. Automated Data Fabric and Data Mesh Aspects   
18  15. Data Governance in the context of Data Fab...   
19                        16. Sample Vendor Offerings   
20       17. Data Fabric and Data Mesh Research Areas   
21  CoK A Survey of Privacy Challenges in Relation...   
22  Converging Data Mesh and Microservice Principl...   
23  Data Integration Revitalized From Data Warehou...   
24  Data Mesh as Distributed Data Platform for Lar...   
25          A Comparative Study of Data Architectures   
26  Cow Mesh a data-mesh architecture to unify dai...   
27      Data Mesh a Systematic Gray Literature Review   
28  Data Mesh Motivational Factors Challenges and ...   
29  From Data Mess to Data Mesh Solution for Futur...   
30  Identifying Alternatives and Deciding Factors ...   
31         Navigating the Data Architecture Landscape   
32  A Distributed Data Mesh Paradigm for an Event-...   
33  Data Mesh Concepts and Principles of a Paradig...   
34  Data Mesh A Holistic Examination Of Its Princi...   
35  Introducing Data Mesh paradigm for Smart City ...   
36  Towards avoiding the data mess Industry insigh...   
37  Utilization of Data Mesh Framework as a Part o...   

                                              Content      Publisher  \
0   abstract data localization law becoming make h...           IEEE   
1   abstract data socio technical decentralized an...           IEEE   
2   abstract enterprise experience exponential gro...           IEEE   
3   abstract paper present systematic ofcommon ana...           IEEE   
4   look back data architecture developed response...       Springer   
5   term data fabric data often viewed different c...       Springer   
6   organization realized importance implementing ...       Springer   
7   defining data fabric underlying data architect...       Springer   
8   introduces key capability concept knowledge ca...       Springer   
9   ai ml dlbroadly speaking ai simulate human per...       Springer   
10  today data rich enterprise frequently confront...       Springer   
11  data describe core entity referred master data...       Springer   
12  ai lifecycleai software engineering fundamenta...       Springer   
13  provide high overview data fabric data evoluti...       Springer   
14  saw data fabric architecture evolution previou...       Springer   
15  look intersection two initiative digital trans...       Springer   
16  metadata managementin digital era enterprise k...       Springer   
17  applying ai metadata intelligent cataloging da...       Springer   
18  data constantly generated live data incredible...       Springer   
19  goal data fabric provide flexible seamless aut...       Springer   
20  must become obvious book unified data fabric a...       Springer   
21  growing volume data appear multiple dis tribut...       Springer   
22  company invested driven design sup port data d...       Springer   
23  year data integration architecture evolved fro...       Springer   
24  rapid increase da

In [3]:
# def generate_custom_token_pattern(ngram_min, ngram_max):
#     # Ensure no word repetition in n-grams
#     word_pattern = r'\b(\w+)\b(?=.*\b\1\b)'
    
#     # Combine word patterns for specified n-gram range
#     ngram_pattern = r'(?:' + word_pattern + r'\s){' + str(ngram_min - 1) + r',' + str(ngram_max - 1) + r'}' + word_pattern
    
#     return ngram_pattern

# token_pattern = generate_custom_token_pattern(1, 4)

In [3]:
def extract_topics(text, tfidf_vectorizer, lda_model):
    
    # TF-IDF Vectorization for the specific document
    tfidf_data = tfidf_vectorizer.transform([text])
    tfidf_values = tfidf_data.toarray()
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    df_tfidf = pd.DataFrame(data=tfidf_values, columns=feature_names)

    # Apply LDA
    doc_topic_prob = lda_model.transform(tfidf_data)

    # Get the most probable topic for the document
    most_probable_topic = np.argmax(doc_topic_prob)

    # Get the top words for the most probable topic with TF-IDF values
    topic = lda_model.components_[most_probable_topic]
    top_keywords_idx = topic.argsort()[:-10 - 1:-1]
    top_keywords = [feature_names[i] for i in top_keywords_idx]
    top_tfidf_values = [tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]
    topic_words = list(zip(top_keywords, top_tfidf_values))

    return most_probable_topic + 1, topic_words


In [4]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')
tfidf_data = tfidf_vectorizer.fit_transform(df['Content'])

# Apply LDA
lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(tfidf_data)

# Assuming 'df' is your DataFrame with 'pdf_content' column
df['topic'], df['topic_words'] = zip(*df.apply(lambda row: extract_topics(row['Content'], tfidf_vectorizer, lda), axis=1))

In [5]:
df

Filename  \
0   Breaking Down Data Silos Data Mesh to Achieve ...   
1   Decentralized Data Governance as Part of a Dat...   
2   Enterprise Data Strategy A Decentralized Data ...   
3   Finding Your Way Through the Jungle of Big Dat...   
4                  01. Evolution of Data Architecture   
5           02. Terminology Data Fabric and Data Mesh   
6    03. Data Fabric and Data Mesh Use Case Scenarios   
7     04. Data Fabric and Data Mesh Business Benefits   
8      05. Key Data Fabric and Data Mesh Capabilities   
9                     06. Relevant ML and DL Concepts   
10      07. AI and ML for a Data Fabric and Data Mesh   
11                       08. AI for Entity Resolution   
12  09. Data Fabric and Data Mesh for the AI Lifec...   
13              10. Data Fabric Architecture Patterns   
14  11. Data Fabric Within an Enterprise Architecture   
15  12. Data Fabric and Data Mesh in a Hybrid Clou...   
16  13. Intelligent Cataloging and Metadata Manage...   
17    14. Automated Data Fabric and Data Mesh Aspects   
18  15. Data Governance in the context of Data Fab...   
19                        16. Sample Vendor Offerings   
20       17. Data Fabric and Data Mesh Research Areas   
21  CoK A Survey of Privacy Challenges in Relation...   
22  Converging Data Mesh and Microservice Principl...   
23  Data Integration Revitalized From Data Warehou...   
24  Data Mesh as Distributed Data Platform for Lar...   
25          A Comparative Study of Data Architectures   
26  Cow Mesh a data-mesh architecture to unify dai...   
27      Data Mesh a Systematic Gray Literature Review   
28  Data Mesh Motivational Factors Challenges and ...   
29  From Data Mess to Data Mesh Solution for Futur...   
30  Identifying Alternatives and Deciding Factors ...   
31         Navigating the Data Architecture Landscape   
32  A Distributed Data Mesh Paradigm for an Event-...   
33  Data Mesh Concepts and Principles of a Paradig...   
34  Data Mesh A Holistic Examination Of Its Princi...   
35  Introducing Data Mesh paradigm for Smart City ...   
36  Towards avoiding the data mess Industry insigh...   
37  Utilization of Data Mesh Framework as a Part o...   

                                              Content      Publisher  \
0   abstract data localization law becoming make h...           IEEE   
1   abstract data socio technical decentralized an...           IEEE   
2   abstract enterprise experience exponential gro...           IEEE   
3   abstract paper present systematic ofcommon ana...           IEEE   
4   look back data architecture developed response...       Springer   
5   term data fabric data often viewed different c...       Springer   
6   organization realized importance implementing ...       Springer   
7   defining data fabric underlying data architect...       Springer   
8   introduces key capability concept knowledge ca...       Springer   
9   ai ml dlbroadly speaking ai simulate human per...       Springer   
10  today data rich enterprise frequently confront...       Springer   
11  data describe core entity referred master data...       Springer   
12  ai lifecycleai software engineering fundamenta...       Springer   
13  provide high overview data fabric data evoluti...       Springer   
14  saw data fabric architecture evolution previou...       Springer   
15  look intersection two initiative digital trans...       Springer   
16  metadata managementin digital era enterprise k...       Springer   
17  applying ai metadata intelligent cataloging da...       Springer   
18  data constantly generated live data incredible...       Springer   
19  goal data fabric provide flexible seamless aut...       Springer   
20  must become obvious book unified data fabric a...       Springer   
21  growing volume data appear multiple dis tribut...       Springer   
22  company invested driven design sup port data d...       Springer   
23  year data integration architecture evolved fro...       Springer   
24  rapid increase da

In [6]:
result_data = []

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Filename = row['Filename']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data.append({'Filename': Filename, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df = pd.DataFrame(result_data, columns=['Filename', 'topic_word', 'idf_value'])

# Print the result DataFrame
result_df

Filename  \
0    Breaking Down Data Silos Data Mesh to Achieve ...   
1    Breaking Down Data Silos Data Mesh to Achieve ...   
2    Breaking Down Data Silos Data Mesh to Achieve ...   
3    Breaking Down Data Silos Data Mesh to Achieve ...   
4    Breaking Down Data Silos Data Mesh to Achieve ...   
..                                                 ...   
375  Utilization of Data Mesh Framework as a Part o...   
376  Utilization of Data Mesh Framework as a Part o...   
377  Utilization of Data Mesh Framework as a Part o...   
378  Utilization of Data Mesh Framework as a Part o...   
379  Utilization of Data Mesh Framework as a Part o...   

                 topic_word  idf_value  
0              localization   3.970414  
1         data localization   3.970414  
2                     smart   2.178655  
3          smart monitoring   3.970414  
4                monitoring   2.178655  
..                      ...        ...  
375                      id   2.871802  
376                  identi   2.871802  
377  relevant ml dl concept   3.970414  
378          relevant ml dl   3.970414  
379           ml dl concept   3.970414  

[380 rows x 3 columns]

In [23]:
# Select the top terms for each PDF
top_terms_df = result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['idf_value'] = top_terms_df['idf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 3, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda = (heatmap + text).properties(width=2000)


In [24]:
# save(chart_lda, "lda_idf_heatmap.html")
chart_lda

alt.LayerChart(...)

In [9]:
result_data_publisher = []

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publisher.append({'Publisher': Publisher, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df_publisher = pd.DataFrame(result_data_publisher)


result_df_publisher

Publisher              topic_word  idf_value
0             IEEE            localization   3.970414
1             IEEE       data localization   3.970414
2             IEEE                   smart   2.178655
3             IEEE        smart monitoring   3.970414
4             IEEE              monitoring   2.178655
..             ...                     ...        ...
375  miscellaneous                      id   2.871802
376  miscellaneous                  identi   2.871802
377  miscellaneous  relevant ml dl concept   3.970414
378  miscellaneous          relevant ml dl   3.970414
379  miscellaneous           ml dl concept   3.970414

[380 rows x 3 columns]

In [10]:
# Select the top terms for each PDF
top_terms_df = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['idf_value'] = top_terms_df['idf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 3.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

alt.LayerChart(...)

## USING ONLY TFIDF

In [12]:
# save(chart_lda_publisher, 'lda_idf_heatmap_pub.html')

In [11]:
def find_top_words_for_each_document(df, num_top_words=10):

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2, 4), token_pattern=r'\b\w+\b')

    # Fit and transform the PDF content
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

    # Get feature names (words) from the TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame with TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on TF-IDF scores
    top_words_for_each_document = []
    for i, row in enumerate(tfidf_df.iterrows()):
        _, document_tfidf_scores = row
        top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with 'pdf_content' and 'top_words'
    result_df = pd.DataFrame({'Filename': df['Filename'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

    return result_df

In [12]:
# Call the function
result_df_2 = find_top_words_for_each_document(df)

# Display the result DataFrame
result_df_2['Publisher'] = df['Publisher']
result_df_2

Filename  \
0   Breaking Down Data Silos Data Mesh to Achieve ...   
1   Decentralized Data Governance as Part of a Dat...   
2   Enterprise Data Strategy A Decentralized Data ...   
3   Finding Your Way Through the Jungle of Big Dat...   
4                  01. Evolution of Data Architecture   
5           02. Terminology Data Fabric and Data Mesh   
6    03. Data Fabric and Data Mesh Use Case Scenarios   
7     04. Data Fabric and Data Mesh Business Benefits   
8      05. Key Data Fabric and Data Mesh Capabilities   
9                     06. Relevant ML and DL Concepts   
10      07. AI and ML for a Data Fabric and Data Mesh   
11                       08. AI for Entity Resolution   
12  09. Data Fabric and Data Mesh for the AI Lifec...   
13              10. Data Fabric Architecture Patterns   
14  11. Data Fabric Within an Enterprise Architecture   
15  12. Data Fabric and Data Mesh in a Hybrid Clou...   
16  13. Intelligent Cataloging and Metadata Manage...   
17    14. Automated Data Fabric and Data Mesh Aspects   
18  15. Data Governance in the context of Data Fab...   
19                        16. Sample Vendor Offerings   
20       17. Data Fabric and Data Mesh Research Areas   
21  CoK A Survey of Privacy Challenges in Relation...   
22  Converging Data Mesh and Microservice Principl...   
23  Data Integration Revitalized From Data Warehou...   
24  Data Mesh as Distributed Data Platform for Lar...   
25          A Comparative Study of Data Architectures   
26  Cow Mesh a data-mesh architecture to unify dai...   
27      Data Mesh a Systematic Gray Literature Review   
28  Data Mesh Motivational Factors Challenges and ...   
29  From Data Mess to Data Mesh Solution for Futur...   
30  Identifying Alternatives and Deciding Factors ...   
31         Navigating the Data Architecture Landscape   
32  A Distributed Data Mesh Paradigm for an Event-...   
33  Data Mesh Concepts and Principles of a Paradig...   
34  Data Mesh A Holistic Examination Of Its Princi...   
35  Introducing Data Mesh paradigm for Smart City ...   
36  Towards avoiding the data mess Industry insigh...   
37  Utilization of Data Mesh Framework as a Part o...   

                                              Content  \
0   abstract data localization law becoming make h...   
1   abstract data socio technical decentralized an...   
2   abstract enterprise experience exponential gro...   
3   abstract paper present systematic ofcommon ana...   
4   look back data architecture developed response...   
5   term data fabric data often viewed different c...   
6   organization realized importance implementing ...   
7   defining data fabric underlying data architect...   
8   introduces key capability concept knowledge ca...   
9   ai ml dlbroadly speaking ai simulate human per...   
10  today data rich enterprise frequently confront...   
11  data describe core entity referred master data...   
12  ai lifecycleai software engineering fundamenta...   
13  provide high overview data fabric data evoluti...   
14  saw data fabric architecture evolution previou...   
15  look intersection two initiative digital trans...   
16  metadata managementin digital era enterprise k...   
17  applying ai metadata intelligent cataloging da...   
18  data constantly generated live data incredible...   
19  goal data fabric provide flexible seamless aut...   
20  must become obvious book unified data fabric a...   
21  growing volume data appear multiple dis tribut...   
22  company invested driven design sup port data d...   
23  year data integration architecture evolved fro...   
24  rapid increase data volume last decade promote...   
25  digital age quantity data produced globally gr...   
26  dairy economically signi cant industry caters ...   
27  world living golden age data idc data corporat...   
28  volume data continues grow organization strivi...   
29  technology advance data volume velocity increa...   
30  data introduced new type data architecture pro... 

In [13]:
result_data_3 = []

# Iterate through each row in the original DataFrame
for _, row in result_df_2.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_3.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_3 = pd.DataFrame(result_data_3)

result_df_3

Filename  \
0    Breaking Down Data Silos Data Mesh to Achieve ...   
1    Breaking Down Data Silos Data Mesh to Achieve ...   
2    Breaking Down Data Silos Data Mesh to Achieve ...   
3    Breaking Down Data Silos Data Mesh to Achieve ...   
4    Breaking Down Data Silos Data Mesh to Achieve ...   
..                                                 ...   
375  Utilization of Data Mesh Framework as a Part o...   
376  Utilization of Data Mesh Framework as a Part o...   
377  Utilization of Data Mesh Framework as a Part o...   
378  Utilization of Data Mesh Framework as a Part o...   
379  Utilization of Data Mesh Framework as a Part o...   

                 topic_word  tfidf_value  
0         data localization     0.642928  
1          data aggregation     0.080145  
2         potential benefit     0.071480  
3         localization data     0.065605  
4    data localization data     0.062981  
..                      ...          ...  
375       data architecture     0.050955  
376     ubiquitous language     0.049306  
377           data pipeline     0.048766  
378          data framework     0.048516  
379     conformed dimension     0.047280  

[380 rows x 3 columns]

In [14]:
# Select the top terms for each PDF
top_terms_df = result_df_3.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['tfidf_value'] = top_terms_df['tfidf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=2000)


In [15]:
# save(chart_tfidf, 'tfidf_heatmap.html')
chart_tfidf

alt.LayerChart(...)

In [16]:
result_data_publisher_2 = []

# Iterate through each row in the original DataFrame
for _, row in result_df_2.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_publisher_2.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_publisher_2 = pd.DataFrame(result_data_publisher_2)


result_df_publisher_2

Publisher                top_word  tf_idf_value
0             IEEE       data localization      0.642928
1             IEEE        data aggregation      0.080145
2             IEEE       potential benefit      0.071480
3             IEEE       localization data      0.065605
4             IEEE  data localization data      0.062981
..             ...                     ...           ...
375  miscellaneous       data architecture      0.050955
376  miscellaneous     ubiquitous language      0.049306
377  miscellaneous           data pipeline      0.048766
378  miscellaneous          data framework      0.048516
379  miscellaneous     conformed dimension      0.047280

[380 rows x 3 columns]

In [17]:
result_df_publisher_2.describe()

tf_idf_value
count    380.000000
mean       0.098316
std        0.061345
min        0.035554
25%        0.060713
50%        0.078689
75%        0.118323
max        0.642928

In [18]:
# Select the top terms for each PDF
top_terms_df_2 = result_df_publisher_2.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_2['tf_idf_value'] = top_terms_df_2['tf_idf_value'] + np.random.rand(top_terms_df_2.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_2).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf_publisher_2 = (heatmap + text).properties(width=1500)
chart_tfidf_publisher_2

alt.LayerChart(...)

In [31]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

In [ ]:
'